## GPT를 활용하여 식물의 상태 분석하기
- 식물의 기본 정보와 분석 데이터를 Chat GPT에 전송하여, 미리 정의된 출력 형식에 맞춘 응답을 받는다.
  - 인풋 데이터 : 1차 분석결과, 식물 환경 센서 데이터, 사용자가 입력한 식물 환경 정보 등
    - 1차 분석결과: 식물의 사진을 식물 질병 예측 모델(CNN)에 입력하여 식물의 질병을 예측한다.
    - 식물 환경 센서 데이터 : 라즈베리파이로 측정한 센서 데이터 값
    - 사용자가 입력한 식물 환경 정보 : 사용자가 직접 입력한 식물의 상태

  - output 형식 : json 형태
    - 구현 방식
      - 방식 1 : Text generation의 JSON mode 사용
      - 방식 2 : Function calling 사용

    

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install openai
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


### 방식 1 : Text generation의 JSON mode

#### 처리 순서
1. 인풋 데이터 텍스트화
2. Chat GPT API 요청
  - 사전 준비 과정
    - 0. GPT API 초기 설정
    - 1. 어시스턴트 생성 및 관리
    - 2. 어시스턴트 쓰레드 생성
    - 3. 어시스턴트 메시지 추가
    - 4. 어시스턴트 실행
3. 응답 파싱 및 번역

\* 인풋데이터는 원시데이터의 형태가 아닌 1차 가공된(날짜별 평균 및 최저 데이터) 데이터로 사전 작업해줘야한다.

##### 1. 인풋 데이터 텍스트화
- 인풋 데이터를 텍스트화 한다.
  - 인풋 데이터 : 1차 분석결과, 식물 환경 센서 데이터, 사용자가 입력한 식물 환경 정보 등
  - 해당 노트북에선 더미 데이터를 만들어 사용한다.

In [5]:
# 더미데이터

# 1차 분석 결과 데이터
predicted_disease = "healthy"

# 식물 환경 센서 데이터
env_data = {
  "2024-06-19": {
    "highest temperature": 41,
    "lowest temperature": 31,
    "average temperature": 38,
    "average humidity": "62%",
    "average illuminance": "7500 lux"
  },
  "2024-06-20": {
    "highest temperature": 42,
    "lowest temperature": 32,
    "average temperature": 39,
    "average humidity": "63%",
    "average illuminance": "7600 lux"
  },
  "2024-06-21": {
    "highest temperature": 43,
    "lowest temperature": 33,
    "average temperature": 40,
    "average humidity": "64%",
    "average illuminance": "7700 lux"
  },
  "2024-06-22": {
    "highest temperature": 44,
    "lowest temperature": 34,
    "average temperature": 41,
    "average humidity": "65%",
    "average illuminance": "7800 lux"
  },
  "2024-06-23": {
    "highest temperature": 45,
    "lowest temperature": 35,
    "average temperature": 42,
    "average humidity": "66%",
    "average illuminance": "7900 lux"
  },
  "2024-06-24": {
    "highest temperature": 46,
    "lowest temperature": 36,
    "average temperature": 43,
    "average humidity": "67%",
    "average illuminance": "8000 lux"
  },
  "2024-06-25": {
    "highest temperature": 47,
    "lowest temperature": 37,
    "average temperature": 44,
    "average humidity": "68%",
    "average illuminance": "8100 lux"
  },
  "2024-06-26": {
    "highest temperature": 48,
    "lowest temperature": 38,
    "average temperature": 45,
    "average humidity": "69%",
    "average illuminance": "8200 lux"
  },
  "2024-06-27": {
    "highest temperature": 49,
    "lowest temperature": 39,
    "average temperature": 46,
    "average humidity": "70%",
    "average illuminance": "8300 lux"
  },
  "2024-06-28": {
    "highest temperature": 50,
    "lowest temperature": 40,
    "average temperature": 47,
    "average humidity": "71%",
    "average illuminance": "8400 lux"
  }
}

# 사용자가 입력한 식물 환경 정보
user_input = {
  "Watering per day": "500ml",
  "wind hours per day": 2,
  "lighting hours per day": 3
}

In [6]:
def data_to_text(predicted_disease, env_data, user_input):
  # "다음 정보를 활용하여 식물을 분석해줘"
  result = "Analyze your plant using the following information.\n"

  # 1차 분석 결과 데이터
  result += f"\n1. Predicted disease: {predicted_disease}\n"

  # 식물 환경 센서 데이터
  result += f"\n2. Environment sensor data: \n"
  for date, sensor_data in env_data.items():
    result += f"Date: {date}\n"
    for key, value in sensor_data.items():
      result += f"- {key}: {value}\n"

  # 사용자가 입력한 식물 환경 정보
  result += f"\n3. User input: \n"
  for key, value in user_input.items():
    result += f"- {key}: {value}\n"

  return result

In [7]:
query=data_to_text(predicted_disease, env_data, user_input)
query

'Analyze your plant using the following information.\n\n1. Predicted disease: healthy\n\n2. Environment sensor data: \nDate: 2024-06-19\n- highest temperature: 41\n- lowest temperature: 31\n- average temperature: 38\n- average humidity: 62%\n- average illuminance: 7500 lux\nDate: 2024-06-20\n- highest temperature: 42\n- lowest temperature: 32\n- average temperature: 39\n- average humidity: 63%\n- average illuminance: 7600 lux\nDate: 2024-06-21\n- highest temperature: 43\n- lowest temperature: 33\n- average temperature: 40\n- average humidity: 64%\n- average illuminance: 7700 lux\nDate: 2024-06-22\n- highest temperature: 44\n- lowest temperature: 34\n- average temperature: 41\n- average humidity: 65%\n- average illuminance: 7800 lux\nDate: 2024-06-23\n- highest temperature: 45\n- lowest temperature: 35\n- average temperature: 42\n- average humidity: 66%\n- average illuminance: 7900 lux\nDate: 2024-06-24\n- highest temperature: 46\n- lowest temperature: 36\n- average temperature: 43\n- a

##### 2. Chat GPT API 요청
- 처리 과정
    - 0. GPT API 초기 설정
    - 1. 어시스턴트 생성 및 관리
    - 2. 어시스턴트 쓰레드 생성
    - 3. 어시스턴트 메시지 추가
    - 4. 어시스턴트 실행

###### 0. GPT API 초기 설정
- API 키 불러오기
- Open AI 객체 생성

In [8]:
# env파일 로드
from dotenv import load_dotenv

dotenv_path = '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/env/gpt.env'
# verbose는 함수 작동시 추가정보 제공의 여부이다.
load_dotenv(dotenv_path, verbose=True)

True

In [9]:
import os
from openai import OpenAI

api_key = os.getenv('GPT_API_KEY')
organization = os.getenv('ORG_ID')
# project_id = os.getenv('PROJECT_ID')

client = OpenAI(
  organization=organization,
  api_key=api_key,
  # project = project_id
)

###### 1. 어시스턴트 생성 및 관리
- 생성
  - JSON mode 설정
  - 출력 포멧 설정
  
  \* 어시스턴트에서는 최대 토큰을 제한할수없다 -> 실행단에서 제어
- 삭제
- 목록 보기

In [11]:
# 어시스턴트 일괄 삭제
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
for assistant in my_assistants.data:
  client.beta.assistants.delete(assistant.id)

In [20]:
assistant = client.beta.assistants.create(
  name="이종현",
  instructions="you are an plant expert. Analyze the condition of the plant based on the entered plant information",
  model="gpt-3.5-turbo",
  response_format = {"type": "json_object"}
)

In [21]:
# 생성된 어시스턴트 목록 보기
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
for assistant in my_assistants.data:
  print(assistant.name)

이종현


###### 2. 어시스턴트 쓰레드 생성 및 관리
- 생성
  - 쓰레드 생성 시 기본 메시지를 추가하여 반환한 JSON형식을 미리 정의해준다.
- 목록 보기 ( GPT API에서 아직 지원을 안한다. -> 자동 삭제)

In [13]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "assistant",
      "content": "Are you a plant expert. Response in json format with Answer 'Crop condition',\
      'Current Status','Improvement plan'.\n 'Crop conditions' only have range from 0 to 5. 0 means the plant is healthy and 5 means\
       the plant is in very critical condition.\n 'Current Status' expresses the current crop status in string form. \
       'improvement plan' expresses the measures necessary for crops to grow healthily in string form."
    },
  ]
)

print(thread)

Thread(id='thread_sLrYqGCYXogangdO7Dfu8dMp', created_at=1719113164, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [14]:
print(thread.id)

thread_sLrYqGCYXogangdO7Dfu8dMp


###### 3. 어시스턴트 메시지 추가 및 관리
- 생성
  - 생성했던 질의문을 쓰레드에 추가한다
- 목록 보기
- 삭제

In [15]:
# 메시지 추가
message = client.beta.threads.messages.create(
  thread.id, # 쓰레드 id
  role="user",
  content=query,
)

In [16]:
# 쓰레드내의 메시지 객체 생성
thread_message = client.beta.threads.messages.list(thread.id)
thread_message

SyncCursorPage[Message](data=[Message(id='msg_ycGSnjOGOv12GQeynLLM8I5C', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Analyze your plant using the following information.\n\n1. Predicted disease: healthy\n\n2. Environment sensor data: \nDate: 2024-06-19\n- highest temperature: 41\n- lowest temperature: 31\n- average temperature: 38\n- average humidity: 62%\n- average illuminance: 7500 lux\nDate: 2024-06-20\n- highest temperature: 42\n- lowest temperature: 32\n- average temperature: 39\n- average humidity: 63%\n- average illuminance: 7600 lux\nDate: 2024-06-21\n- highest temperature: 43\n- lowest temperature: 33\n- average temperature: 40\n- average humidity: 64%\n- average illuminance: 7700 lux\nDate: 2024-06-22\n- highest temperature: 44\n- lowest temperature: 34\n- average temperature: 41\n- average humidity: 65%\n- average illuminance: 7800 lux\nDate: 2024-06-23\n- highest temperature: 45\n- lowest temperature: 35\n-

In [17]:
# 각 메시지 출력
for index,message in enumerate(thread_message.data[::-1]):
  print(f"====================== {index+1}번째 질문 ==========================")
  print("역할:",message.role)
  print(message.id)
  print(message.content[0].text.value)
  print("============================================================================")

====================== 1번째 질문 ==========================
역할: assistant
msg_XJ64WEd2or8S4XiTyhQPc7SE
Are you a plant expert. Response in json format with Answer 'Crop condition',      'Current Status','Improvement plan'.
 'Crop conditions' only have range from 0 to 5. 0 means the plant is healthy and 5 means       the plant is in very critical condition.
 'Current Status' expresses the current crop status in string form.        'improvement plan' expresses the measures necessary for crops to grow healthily in string form.
====================== 2번째 질문 ==========================
역할: user
msg_ycGSnjOGOv12GQeynLLM8I5C
Analyze your plant using the following information.

1. Predicted disease: healthy

2. Environment sensor data: 
Date: 2024-06-19
- highest temperature: 41
- lowest temperature: 31
- average temperature: 38
- average humidity: 62%
- average illuminance: 7500 lux
Date: 2024-06-20
- highest temperature: 42
- lowest temperature: 32
- average temperature: 39
- average humidity: 6

###### 4. 어시스턴트 실행
- 어시스턴트와 쓰레드를 사용하여 GPT에서 답변을 응답 받는다.

- 어시스턴트 실행에선 추가적으로 instructions을 설정할 수 있다.
  - 보통 사용자의 이름을 지정하거나 추가정보를 기입한다.

In [22]:
run = client.beta.threads.runs.create(
  thread_id= thread.id,
  assistant_id= assistant.id,
  instructions="The user's name is 'LEE JONGHYEON'.",
  # max_prompt_tokens= 600 # 옵션
)

print(run)

Run(id='run_kemyrCzJpl7sYoQXmA4g5Y8S', assistant_id='asst_7k2eDGmB5gplqJ8OHVik2im0', cancelled_at=None, completed_at=None, created_at=1719113224, expires_at=1719113824, failed_at=None, incomplete_details=None, instructions="The user's name is 'LEE JONGHYEON'.", last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format=AssistantResponseFormat(type='json_object'), started_at=None, status='queued', thread_id='thread_sLrYqGCYXogangdO7Dfu8dMp', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [65]:
# 출력 결과 확인
thread_message = client.beta.threads.messages.list(thread.id)
print(thread_message.data[0].content[0].text.value)
response_string = thread_message.data[0].content[0].text.value

{
    "Crop condition": 0,
    "Current Status": "The plant is healthy and not showing any signs of disease. The environmental conditions are optimal for growth, with appropriate temperatures, humidity, and illuminance levels.",
    "Improvement plan": "Since the plant is predicted to be healthy, continue to monitor the environmental conditions closely and maintain the current watering, wind exposure, and lighting schedule. Regularly inspect the plant for any signs of pests or diseases and take appropriate action if needed."
}


##### 3. 응답 파싱 및 번역
- 문자열을 json 데이터 타입으로 저장히기 위해 파싱한다.
- GCP Translation API을 활용하여 번역한다.

###### JSON 형식으로 파싱

In [66]:
import json

response_json = json.loads(response_string)
response_json

{'Crop condition': 0,
 'Current Status': 'The plant is healthy and not showing any signs of disease. The environmental conditions are optimal for growth, with appropriate temperatures, humidity, and illuminance levels.',
 'Improvement plan': 'Since the plant is predicted to be healthy, continue to monitor the environmental conditions closely and maintain the current watering, wind exposure, and lighting schedule. Regularly inspect the plant for any signs of pests or diseases and take appropriate action if needed.'}

###### GCP Translation API 인증 및 초기 설정
- 라이브러리 설치
  - google.cloud.translate : 구글 번역을 쓸 수 있도록 처리해주는 라이브러리
  

In [1]:
!pip install google-cloud-translate

###### GCP API 키 로드 및 환경변수 설정

In [42]:
# env파일 로드
from dotenv import load_dotenv

dotenv_path = '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/env/GCP.env'
# verbose는 함수 작동시 추가정보 제공의 여부이다.
load_dotenv(dotenv_path, verbose=True)

True

In [67]:
api_key = os.getenv('GCP_API_KEY_PATH')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = api_key
project_id = os.getenv('PROJECT_ID')

###### 번역 함수 정의

In [72]:
# Imports the Google Cloud Translation library
from google.cloud import translate

# Initialize Translation client
def translate_text(
    text: str = "YOUR_TEXT_TO_TRANSLATE", project_id: str = "YOUR_PROJECT_ID"
) -> translate.TranslationServiceClient:
    """Translating Text."""

    client = translate.TranslationServiceClient()

    location = "global"

    parent = f"projects/{project_id}/locations/{location}"

    # Translate text from English to French
    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats
    response = client.translate_text(
        request={
            "parent": parent,
            "contents": [text],
            "mime_type": "text/plain",  # mime types: text/plain, text/html
            "source_language_code": "en-US",
            "target_language_code": "ko",
        }
    )

    # Display the translation for each input text provided
    # for translation in response.translations:
    #     print(f"Translated text: {translation.translated_text}")

    return response.translations[0].translated_text

###### Chat GPT 응답을 번역후 저장

In [74]:
response_json['Current Status'] = translate_text(response_json['Current Status'], project_id)
response_json['Improvement plan'] = translate_text(response_json['Improvement plan'], project_id)
response_json

{'Crop condition': 0,
 'Current Status': '식물은 건강하며 질병의 징후도 보이지 않습니다. 환경 조건은 적절한 온도, 습도, 조도 수준으로 성장에 최적입니다.',
 'Improvement plan': '식물이 건강할 것으로 예상되므로 계속해서 환경 조건을 면밀히 모니터링하고 현재의 물 공급, 바람 노출 및 조명 일정을 유지하십시오. 식물에 해충이나 질병의 징후가 있는지 정기적으로 검사하고 필요한 경우 적절한 조치를 취하십시오.'}